In the following problems, use the data provided in the files in.dta and out.dta for Homework # 6. We are going to apply linear regression with a nonlinear transformation for classification (without regularization). The nonlinear transformation is given by $\phi_0$ through $\phi_7$ which transform ($x_1$, $x_2$) into

$$1 \quad x_1 \quad x_2 \quad x_1^2 \quad x_2^2 \quad x_1x_2 \quad \left |x_1 - x_2 \right | \quad \left |x_1 + x_2 \right | $$

To illustrate how taking out points for validation affects the performance, we will consider the hypotheses trained on $\mathcal{D}_{\text{train}}$ (without restoring the full $\mathcal{D}$ for training after validation is done).

Q1. Split in.dta into training (first 25 examples) and validation (last 10 examples). Train on the 25 examples only, using the validation set of 10 examples to select between five models that apply linear regression to $\phi_0$ through $\phi_k$, with $k = 3, 4, 5, 6, 7$. For which model is the classification error on the validation set smallest?

In [172]:
import numpy as np
import requests

# get data
train_req = requests.get("http://work.caltech.edu/data/in.dta")
train = np.array([row.split() for row in train_req.text.split("\r\n")[:-1]]).astype(float)

test_req = requests.get("http://work.caltech.edu/data/out.dta")
test = np.array([row.split() for row in test_req.text.split("\r\n")[:-1]]).astype(float)

In [168]:
def trainValidate(training, validating, testing):
    # using validation for determining number of features to use
    val_errors = np.zeros(shape = (5,2))
    out_errors = np.zeros(shape = 5)
    for k in range(3, 8):
        val_test = LinearRegression(training, validating, k)
        val_test.regress()
        val_errors[k-3] = val_test.classificationError()

        out_errors[k-3] = val_test.outOfSampleError(testing)


    # get index of min of validation and out error, add 3 to get k
    return (val_errors, out_errors)

In [173]:
# split data into train and validation 
validate = train[-10:, :]
train = train[:25, :]

(val_errors, out_errors) = trainValidate(train, validate, test)


print("Smallest validation error at k = {}.".format(np.argmin(val_errors[:, 1]) + 3))
print("Smallest out of sample error at k = {}.".format(np.argmin(out_errors) + 3))

Smallest validation error at k = 6.
Smallest out of sample error at k = 7.


In [177]:
print(val_errors)
print(out_errors)

[[ 0.44  0.3 ]
 [ 0.32  0.5 ]
 [ 0.08  0.2 ]
 [ 0.04  0.  ]
 [ 0.04  0.1 ]]
[ 0.42   0.416  0.188  0.084  0.072]


__ A1. [d] k = 6 __

Q2. Evaluate the out-of-sample classification error using out.dta on the 5 models
to see how well the validation set predicted the best of the 5 models. For which
model is the out-of-sample classification error smallest?

__A2. [e] k = 7 __

Q3. Reverse the role of training and validation sets; now training with the last 10 examples and validating with the first 25 examples. For which model is the classification error on the validation set smallest?

In [175]:
# repeat but swap train and validate
train = np.array([row.split() for row in train_req.text.split("\r\n")[:-1]]).astype(float)
test = np.array([row.split() for row in test_req.text.split("\r\n")[:-1]]).astype(float)

validate_swap = train[:25, :]
train_swap = train[-10:, :]
(val_errors_swap, out_errors_swap) = trainValidate(train_swap, validate_swap, test)
print("Smallest validation error at k = {}.".format(np.argmin(val_errors_swap[:, 1]) + 3))
print("Smallest out of sample error at k = {}.".format(np.argmin(out_errors_swap) + 3))

Smallest validation error at k = 6.
Smallest out of sample error at k = 6.


__A3. [d] k = 6__

Q4. Once again, evaluate the out-of-sample classification error using out.dta on the 5 models to see how well the validation set predicted the best of the 5 models. For which model is the out-of-sample classification error smallest?

__A4. [d] k = 6 __

Q5. What values are closest in Euclidean distance to the out-of-sample classification error obtained for the model chosen in Problems 1 and 3, respectively?

In [176]:
print(out_errors[np.argmin(out_errors)])
print(out_errors_swap[np.argmin(out_errors)])

0.072
0.196


__ A5. [b] 0.1 0.2 __

Q6. Let $e_1$ and $e_2$ be independent random variables, distributed uniformly over the interval $[0, 1]$. Let $e = \text{min}(e_1, e_2)$. The expected values of $e_1$, $e_2$, $e$ are closest to

A6. Let $X = \text{min} (e_1, e_2)$; to solve for its expected value, we first need its CDF,

$$F(x) = \mathbb{P}(X \leq x) = 1 - \mathbb{P}(X > x) = 1 - \mathbb{P}(\min(e_1, e_2) > x)$$

Clearly the probability that the minimum of two variables is greater than $x$ is the joint probability that both variables are greater than $x$,

$$F(x) = 1 - \mathbb{P}(e_1 > x, e_2 > x) $$

and because of independence, 

$$F(x) = 1 - \mathbb{P}(e_1 > x) \mathbb{P} (e_2 > x)  = 1 - \mathbb{P}(e_1 > x)^2$$

and where, because $e_1$ is uniformly distributed over $[0,1]$, we have,

$$F(x) = 1 - \left ( \frac{x - 1}{0 - 1} \right )^2 = 1 - (1-x)^2 $$

differentiating to get the density, 

$$ f(x) = 2(1-x) $$

and then finally using the pdf to comptue the expected value, 

$$ \mathbb{E}[x] = \int_{0}^{1} x f(x) dx = 2 \int_{0}^{1} x - x^2 dx = 2 \left [ \frac{x^2}{2} - \frac{x^3}{3} \right ]_{0}^{1}  =  2(1 / 2 - 1/3) = 1/3 = 0.\bar{3}$$





__ A6. [d] 0.5, 0.5, 0.4 __


Q7. You are given the data points $(x, y): (−1, 0),(\rho, 1),(1, 0), \rho \geq 0$, and a choice between two models: constant $\{ h_0(x) = b \}$ and linear $\{ h_1(x) = ax + b \}$.
For which value of $\rho$ would the two models be tied using leave-one-out cross-validation with the squared error measure?

A7. For the constant model, we just average the $y$s to solve for $b$. For the linear model, we have two points so we can just solve for the line between them. So we choose 2 points to create the model, and use the squared error of the 3rd point to estimate the model and repeat. 

So for the constant model: 



validate = $(-1, 0)$

train = $(\rho, 1), (1, 0)$

So that, $h_1(x) = 0.5$ and $h_1(-1) = 0$, with squared error $0.25$. 

For $h_2$: 

$a = \frac{1}{\rho - 1} $

$b = y - ax = 0 - \frac{1}{\rho - 1} = -\frac{1}{\rho - 1} $

$ h_2(-1) = - \frac{1}{\rho-1} - \frac{1}{\rho-1} = -\frac{2}{\rho -1} $, with squared error $\frac{4}{(\rho - 1)^2}$

Repeat with validate = $(\rho, 1)$

train = $(-1, 0), (1, 0)$

$h_1(x) = 0$ and $h_1(\rho) = 0$, with squared error $1$. 

$h_2$:

$a = \frac{0}{2} = 0$

$b = 0 - 0 = 0$

$h_2(\rho) = 0$ with squared error $1$. 

Finally, with validate point = $(1, 0)$

train = $(-1, 0), (\rho, 1)$

$h_1(x) = 0.5$, $h_1(1) = 0.5$ with squared error $0.25$. 

$h_2$:

$a = \frac{1}{\rho + 1} $

$b = 0 - (- \frac{1}{\rho + 1}) = \frac{1}{\rho+1}$

$h_2(x) = \frac{a}{\rho+1} + \frac{1}{\rho+1}$, $h_2(1) = \frac{2}{\rho+1}$ with squared error, $\frac{4}{(\rho+1)^2} $

So that the squared error, for setting the sum of the squared errors equal, 

$$0.25 + 1 + 0.25 = \frac{4}{(\rho -1)^2} + 1 + \frac{4}{(\rho+1)^2}$$

$$ 0.5 =  \frac{4}{(\rho -1)^2} + \frac{4}{(\rho+1)^2}$$

$$ (\rho^2 - 1)^2 = 8 (\rho + 1)^2 + 8 (\rho - 1)^2$$

$$ \rho^4 - 2\rho^2 + 1 = 8(\rho^2 + 2 \rho + 1) + 8 (\rho^2 - 2\rho + 1)$$

$$ \rho^4 - 2 \rho^2 + 1 = 16\rho^2 + 16 $$

$$ \rho^4 - 18 \rho^2 - 15 = 0$$

$$ \rho = \sqrt{9 + 4 \sqrt{6}} $$ for $\rho \geq 0$

__ [c]  $\sqrt{9 + 4 \sqrt{6}}$ __

_Notice: Quadratic Programming packages sometimes need tweaking and have numerical issues, and this is characteristic of packages you will use in practical ML situations. Your understanding of support vectors will help you get to the correct answers._

In the following problems, we compare PLA to SVM with hard margin on linearly separable data sets. For each run, you will create your own target function $f$ and data set $\mathcal{D}$. Take $d = 2$ and choose a random line in the plane as your target function $f$ (do this by taking two random, uniformly distributed points on $[−1, 1] \times [−1, 1]$
and taking the line passing through them), where one side of the line maps to $+1$ and the other maps to $−1$. Choose the inputs $x_n$ of the data set as random points in $\mathcal{X} = [−1, 1] \times [−1, 1]$, and evaluate the target function on each $x_n$ to get the corresponding output $y_n$. If all data points are on one side of the line, discard the
run and start a new run. 

Start PLA with the all-zero vector and pick the misclassified point for each PLA iteration at random. Run PLA to find the final hypothesis $g_{\text{PLA}}$ and measure the disagreement between $f$ and
$g_{\text{PLA}}$ as $\mathbb{P}[f(x) \neq g_{\text{PLA}}(x)]$ (you can either calculate this exactly, or approximate it by generating a sufficiently large, separate set of points to evaluate it). Now, run SVM on the same data to find the final hypothesis $g_{\text{SVM}}$ by solving

\begin{align}
&\min_{w, b} \quad \frac{1}{2} \mathbf{w}^T \mathbf{w} \\
&s.t. \quad y_n(\mathbf{w}^T \mathbf{x}_n + b) \geq 1 
\end{align}

using quadratic programming on the primal or the dual problem. Measure the disagreement between $f$ and $g_{\text{SVM}}$ as $\mathbb{P}[f(\mathbf{x}) \neq g_{\text{SVM}}(\mathbf{x})]$, and count the number of support vectors you get in each run. 


We can solve the above optimization problem using Lagrange Multipliers; i.e., get the Lagrangian and set its gradient equal to zero then substitute back to the get the dual formulation of the problem:

$$\mathcal{L}(\mathbf{w}, b, \alpha) = \frac{1}{2} \mathbf{w}^T \mathbf{w} - \sum_{i=1}^{N} \alpha_i [ y_i (\mathbf{w}^T \mathbf{x}_i + b) - 1] $$

$$ \nabla \mathcal{L} (\mathbf{w}, b, \alpha) = \mathbf{0} $$

$$ \frac{\partial L}{\partial \mathbf{w}} = \mathbf{w} - \sum_{i=1}^{N} \alpha_i y_i \mathbf{x}_i = 0 $$

or, 

$$\mathbf{w} = \sum_{i}^{N} \alpha_i y_i \mathbf{x}_i $$

Then, 

$$\frac{\partial L}{\partial b} = - \sum_{i=1}^N \alpha_i y_i = 0 $$

or, 

$$ \sum_{i=1}^{N} a_i y_i = 0 $$

Substituting these equations back into the original Lagrangian, 

$$\mathcal{L} = \frac{1}{2} (\sum_{i=1}^{N} \alpha_i y_i \mathbf{x}_i) \cdot (\sum_{j=1}^{N} \alpha_j y_j \mathbf{x}_j) -  \sum_{i=1}^{N} \alpha_i y_i \mathbf{x}_i( \sum_{j=1}^{N} \alpha_j y_j \mathbf{x}_j) - \sum_{i=1}^{N} \alpha_i y_i b + \sum_{i=1}^{N} \alpha_i$$

$$\mathcal{L}(\alpha)  = \sum_{i=1}^{N} \alpha_i - \frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_i \alpha_j y_i y_j \mathbf{x}^T_i \cdot \mathbf{x}_j $$

so that we can maximize with respect only to $\alpha$ with the constraints $\alpha_i \geq 0$ and the previous constraint $\sum_{i=1}^{N} \alpha_i y_i = 0$. 

Then to get this in terms that cvxopt will understand, we need to minimize the negative of this, i.e., 

$$ \min_a \frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} y_i y_j \alpha_i \alpha_j \mathbf{x}_i^T \mathbf{x}_j - \sum_{i = 1}^N \alpha_i $$

So the quadratic coefficients form an $N \times N$ matrix $Q$ with entries $p[i][j] = y_i y_j \mathbf{x}_i^T \mathbf{x}_j $, and the linear coefficients are just a $N$ dimensional vector of $-1$s (with the constraints as above). 

Explicitly, once the QP gives us the alphas, non-zero values represent the indices of support vectors, we can use the alphas to solve for the weights, and then use any support vector to solve for the constant bias $b$, since,

$$y_i (\mathbf{w}^T \mathbf{x}_i + b) = 1 $$
$$y_i \mathbf{w}^T \mathbf{x}_i + y_i b = 1 $$

Since $y_i \in \{ +1, -1 \}$, $1 / y_i = y_i$ and $(y_i)^2 = 1$, so, 

$$ b = \frac{1}{y_i} (1 - y_i \mathbf{w}^T x_i) = y_i - \mathbf{w}^T \mathbf{x}_i$$

In [7]:
import numpy as np
# quadratic programming package
from cvxopt import matrix, solvers

class SVM():
    ''' Support Vector Machine Model compared to Perceptron Learning Algorithm on random points in R2'''
    def __init__(self, N):
        ''' generate target function and N random points; initialize weights'''
        # number of data points
        self.N = N
        
        self.getPoints()
        
        # test if all values are on one side or another of the target
        while np.all(self.y == 1) or np.all(self.y == -1):
            # if so, reinitialize target and points
            self.getPoints()
            
        # initialize weights to zero
        self.w = np.zeros(2)
        
        # initialize bias to zero
        self.b = 0

    def getPoints(self):
         # get two points to determine target function line
        p = np.random.uniform(-1, 1, (2, 2))
        
        # get equation of that line
        m = (p[1][1] - p[0][1]) / (p[1][0] - p[0][0])
        
        b = p[1][1] - m * p[1][0]
        
        # target function/line
        t = lambda x: m * x[0] + b
        
        # target applied to full array
        self.target = lambda x: np.where(np.apply_along_axis(t, 1, x) > 0, 1, -1)
        
        # generate N data points
        self.x = np.random.uniform(-1, 1, (self.N, 2))
        
        # apply target to data to get proper classified ys
        self.y = self.target(self.x)
        
    def PLA_test(self):
        ''' run perceptron learning algorithm with same data, return disagreement between target and hypothesis '''
        pla = PLA(self.N, self.target, self.x, self.y)
        return pla.run()

    
    def SVM_run(self):
        ''' run support vector machine'''
        # formulate data into quadratic programming constraints to get lagrange multipliers
        # gram matrix of data (i.e., matrix of all inner products)
        gram = np.dot(self.x, self.x.T)
        # quadratic terms
        P = matrix(np.outer(self.y,self.y) * gram, tc = 'd')
        # linear terms 
        q = matrix(-np.ones(self.N), tc = 'd')
        # constraints
        A = matrix(self.y, (1,self.N), tc = 'd')
        b = matrix(0.0)
        
        G = matrix(np.diag(np.ones(self.N) * -1), tc = 'd')
        h = matrix(np.zeros(self.N), tc = 'd')
        
        # make cxvopt solver silent 
        solvers.options['show_progress'] = False
        
        # get solution
        sol = solvers.qp(P, q, G, h, A, b)
        
        # Lagrange multipliers
        alpha = np.array(sol['x'])
        
        # boolean mask for support vectors
        sv = np.where(alpha > 10**-4)[0]
        
        # if no support vectors; all points are to one side
        if sv.shape[0] == 0:
            # start over with new data
            print("Init Error: all points to one side. ")
            
        # solve for weights (elementwise multiply alpha*y*x then sum)
        self.w = np.sum((alpha[sv].T * self.y[sv]).T[np.newaxis] * self.x[sv], axis = 1)
            
        # solve for bias using first support vector
        self.b = self.y[sv[0]] - np.inner(self.w, self.x[sv[0]])
        
        # return support vectors
        return self.x[sv]

    def SVM_test(self):
        ''' run SVM, generate testing data, approximate out of sample error'''
        # generate random testing points
        test_x = np.random.uniform(-1, 1, (1000, 2))
        
        # classify with actual target
        test_y = self.target(test_x)
        
        # compare classification with SVM prediction
        return test_y[(test_y != self.classify(test_x))[0]].shape[0] / test_y.shape[0]
        
    def classify(self, data):
        ''' use SVM to predict on data '''
        return np.where(np.inner(self.w, data) > 0, 1, -1)
    
    def compare(self):
        ''' run both algorithms, compare out of sample error approximations '''
        # run SVM/get number of support vectors
        supports = self.SVM_run()
        
        # compare performance of SVM and PLA, return 1 if SVM is better
        if self.SVM_test() > self.PLA_test():
            return (1, supports)
        else:
            return (0, supports)
    
class PLA():
    ''' Perceptron Learning Algorithm (for comparison)'''
    def __init__(self, N, target, data, y):
        self.N = N
        self.target = target
        
        self.x = np.concatenate((np.ones(data.shape[0]).reshape(data.shape[0], 1), data), axis = 1)
        self.y = y
        
        self.w = np.zeros(3)
        
    def h(self, x):
        return np.where(np.inner(self.w.T, x) > 0, 1, -1)

    def update(self):
        ''' apply one iteration of PLA '''
        hyp = self.h(self.x)
        
        # get all misclassified points, multiply by what direction it's wrong in
        missed = (hyp != self.y)
        ys = self.y[missed]
        
        if len(ys) == 0:
            # if nothing misclassified, learning complete
            return True
        else:
            # randomly choose a wrong point, use it to update weights
            wrong_index = np.random.randint(len(ys))
            self.w += ys[wrong_index] * self.x[missed][wrong_index]
            return False
        
    def run(self):
        ''' run PLA to completion, return probability of error '''
        while (not self.update()):
            pass
        
        # generate testing set
        test_x = np.random.uniform(-1, 1, (1000, 2))
        test_y = self.target(test_x)
        
        test_x = np.concatenate((np.ones(1000).reshape(1000, 1), test_x), axis = 1)
        
        return (test_y[self.h(test_x) != test_y].shape[0]) / test_y.shape[0]

                                
        
        

In [8]:
# counter for number of times SVM beats PLA
gsvm = 0

# run experiment 1000 times
for i in range(1000):
    t = SVM(10)
    # unpack tuple of GSVM and supports (no question for that here)
    (g, _) = t.compare()
    
    gsvm += g

print(gsvm / 1000)

0.712


Q8. For N = 10, repeat the above experiment for 1000 runs. How often is $g_{SVM}$
better than $g_{PLA}$ in approximating $f$? The percentage of time is closest to:

__ A8. [d] 80% __

_should be [c] here apparently, but does not agree_

In [9]:
# same but for N = 100 and counting support vectors
supports = np.empty(1000, dtype = int)

# counter for number of times SVM beats PLA
gsvm = 0

# run experiment 1000 times
for i in range(1000):
    t = SVM(100)
    
    # unpack tuple
    (g, s) = t.compare()
    gsvm += g
    supports[i] = s.shape[0]

print(gsvm / 1000)
print(np.average(supports))

0.96
3.019


Q9. For $N = 100$, repeat the above experiment for $1000$ runs. How often is $g_{SVM}$
better than $g_{PLA}$ in approximating $f$? The percentage of time is closest to:

__ A9. [e] 90% __

Q10. For the case $N = 100$, which of the following is the closest to the average number
of support vectors of $g_{SVM}$ (averaged over the $1000$ runs)?

__ A10. [b] 3__ 

In [161]:
class LinearRegression():
    ''' Linear Regression Learning Model (copied from HW6)'''
    def __init__(self, train, test, k = 2):
        ''' assuming data is formatted so that last row are ys, rest is xs'''
        # number of features to use
        self.k = k+1
        self.x = train[:, :-1]
        
        # add column of 1s to data
        self.x = np.concatenate((np.ones(train.shape[0]).reshape(train.shape[0], 1), self.x), axis =1)
        
        # seperate out ys
        self.y = train[:, -1]
        
        # apply non-linear transform; use k features
        self.x = self.transform(self.x)[:, :self.k]
        
        # initialize weights to 0
        self.w = np.zeros(self.x.shape[1])
        
        # seperate and apply transformation to testing data
        self.test_x = test[:, :-1]
        self.test_x = np.concatenate((np.ones(test.shape[0]).reshape(test.shape[0], 1), self.test_x), axis =1)

        self.test_y = test[:, -1]        
        self.test_x = self.transform(self.test_x)[:, :self.k]
    
    def regress(self, l = 0):
        ''' solve for regularized weights with lambda = l; defaults to no regularization '''
        #self.w = np.linalg.lstsq(np.dot(self.x.T, self.x) + l * np.identity(self.x.shape[1]), np.dot(self.x.T, self.y))[0]
        self.w = np.inner(np.linalg.pinv(self.x), self.y)
        
        
    def classify(self, data):
        ''' hard threshold regression for classification '''
        return np.where(np.inner(self.w, data) > 0, 1.0, -1.0)
    
    def classificationError(self):
        ''' calculate in and out of sample/validation error '''
        ein = self.y[self.classify(self.x) != self.y].shape[0]
        eout = self.test_y[self.classify(self.test_x) != self.test_y].shape[0]
        
        # return fraction of misaclassified points in and out of sample
        return (ein / self.y.shape[0], eout / self.test_y.shape[0])
        
    def outOfSampleError(self, data):
        ''' use seperate data set to estimate out of sample error '''
        out_x = data[:, :-1]
        out_x = np.concatenate((np.ones(data.shape[0]).reshape(data.shape[0], 1), out_x), axis =1)

        out_y = data[:, -1]        
        out_x = self.transform(out_x)[:, :self.k]
        
        return (out_y[self.classify(out_x) != out_y].shape[0]/ out_y.shape[0])
    
    def transform(self, data):
        ''' applies specified nonlinear transform on data by adding columns'''
        trans = self.addFeature(data, np.square(data[:, 1]))
        trans = self.addFeature(trans, np.square(data[:, 2]))
        trans = self.addFeature(trans, data[:, 1]*data[:, 2])
        trans = self.addFeature(trans, np.abs(data[:, 1] - data[:, 2]))
        trans = self.addFeature(trans, np.abs(data[:, 1] + data[:, 2]))

        return trans

    def addFeature(self, data, feature):
        ''' helper function for adding feature '''
        return np.concatenate((data, feature.reshape(data.shape[0], 1)), axis = 1)
    